In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification 
import pandas as pd
import librosa
import numpy as np
from tqdm import tqdm

import sys
sys.path.append("..")
from src.models import Wav2Vec2FacebookFixMatch
from src.datamodules import FixMatchDataset

In [2]:
# model_name_or_path = "facebook/wav2vec2-base"
# feature_extractor = AutoFeatureExtractor.from_pretrained(model_name_or_path)
# sampling_rate = feature_extractor.sampling_rate

path = "/home/work/joono/joono/joono/DV_DV.Deep/tqh0z3f8/checkpoints/best-checkpoint.ckpt"
# model = Wav2Vec2Facebook.load_from_checkpoint(path, args={})
model = Wav2Vec2FacebookFixMatch.load_from_checkpoint(path, args={'args': {}, 'train': False})

/home/work/joono/anaconda3/envs/torch/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/work/joono/anaconda3/envs/torch/lib/python3.12/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizat

======== Trainable Lora layers ===========
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([

In [3]:
# Collate 함수 정의
def collate_fn(batch):
    # signals = zip(*batch)
    signals = batch
    max_length = max([signal.size(0) for signal in signals])
    padded_signals = torch.zeros(len(signals), max_length)
    for i, signal in enumerate(signals):
        padded_signals[i, :signal.size(0)] = signal
    # labels = torch.tensor(labels)
    return padded_signals

In [4]:
test_df = pd.read_csv('../dataset/unlabeled_data.csv', index_col=None)
test_df['path'] = '../dataset/' + test_df['path'].str[1:]
# test_dataset = TestDataset(test_df)
test_dataset = FixMatchDataset(test_df, train_mode=False, model_name='facebook/wav2vec2-base')
# test_loader = DataLoader(test_dataset, shuffle=False, num_workers=24, batch_size=32)
test_loader = DataLoader(test_dataset, shuffle=False, num_workers=24, batch_size=32, collate_fn=collate_fn)

/home/work/joono/anaconda3/envs/torch/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/work/joono/anaconda3/envs/torch/lib/python3.12/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [5]:
def inference(model, test_loader, device):
    model.to(device)
    model = model.eval()
    categories = []
    with torch.no_grad():
        # for inputs, labels in tqdm(test_loader):
        for inputs in tqdm(test_loader):
            
            inputs = inputs.to(device)
            # labels = labels.to(device)
            
            category = model(inputs)

            categories+= category.cpu().detach().numpy().tolist()
            
    return categories

In [6]:
categories = inference(model=model, test_loader=test_loader, device='cuda:1')
# preds = model.inference(test_loader=test_loader)

100%|██████████| 40/40 [00:11<00:00,  3.59it/s]


In [7]:
categories_np = np.array(categories)
categories_class = np.argmax(categories_np, axis=1)
print(len(categories_class))

categories_softmax = np.exp(categories_np) / np.sum(np.exp(categories_np), axis=1, keepdims=True)
print(len(categories_softmax))

category_probs = np.zeros((len(categories_softmax), 2))
category_count = [0, 0, 0, 0]
for row in range(len(category_probs)):
    cat_max_idx = np.argmax(categories_softmax[row])
    category_count[cat_max_idx] += 1
    prob = categories_softmax[row, cat_max_idx]
    
    print(f"cat_max: {cat_max_idx}, prob: {prob}")
    
    if      cat_max_idx == 0: category_probs[row] = np.array([0, 0])
    elif    cat_max_idx == 1: category_probs[row] = np.array([prob, 0])
    elif    cat_max_idx == 2: category_probs[row] = np.array([0, prob])
    elif    cat_max_idx == 3: category_probs[row] = np.array([prob, prob])
        
print(category_count)

1264
1264
cat_max: 2, prob: 0.9993283377310587
cat_max: 2, prob: 0.9993658791190964
cat_max: 2, prob: 0.9992296802337197
cat_max: 2, prob: 0.9996870710708657
cat_max: 2, prob: 0.9996278291529186
cat_max: 2, prob: 0.7600599086092166
cat_max: 2, prob: 0.9995299067867016
cat_max: 2, prob: 0.9997021405229366
cat_max: 2, prob: 0.9996193232448667
cat_max: 2, prob: 0.999679383014734
cat_max: 2, prob: 0.9995799150040369
cat_max: 2, prob: 0.9980293997162887
cat_max: 2, prob: 0.9996104903871962
cat_max: 2, prob: 0.9994309770670832
cat_max: 3, prob: 0.9058886816638066
cat_max: 2, prob: 0.8637450250395962
cat_max: 1, prob: 0.9985597380938164
cat_max: 2, prob: 0.9992778944802144
cat_max: 2, prob: 0.999443494584843
cat_max: 2, prob: 0.999466452908007
cat_max: 2, prob: 0.9996562872625969
cat_max: 2, prob: 0.9995219361737273
cat_max: 2, prob: 0.9929396731153548
cat_max: 2, prob: 0.999354815330613
cat_max: 2, prob: 0.9993582820357221
cat_max: 2, prob: 0.9995884556867951
cat_max: 2, prob: 0.999372678449

In [ ]:
submit = pd.read_csv('/home/work/joono/joono/dataset/sample_submission.csv')

max_thres = 0.99999
min_thres = 0.00001

fake, real = 0, 1

for i in tqdm(range(len(category_probs))):
    if      category_probs[i][fake] > max_thres  : submit.iloc[i, 1] = 1
    elif    category_probs[i][fake] < min_thres  : submit.iloc[i, 1] = 0 
    else                                         : submit.iloc[i, 1] = category_probs[i][0]
    if      category_probs[i][real] > max_thres  : submit.iloc[i, 2] = 1
    elif    category_probs[i][real] < min_thres  : submit.iloc[i, 2] = 0 
    else                                         : submit.iloc[i, 2] = category_probs[i][1]

In [ ]:
submit.to_csv('wav2vec2_train_test_align_lora_fixmatch_test2.csv', index=False)

In [ ]:
preds

In [ ]:
import os
import pathlib
from glob import glob

In [ ]:

ids, paths = [], []
for ul_path in glob("/home/work/joono/joono/dataset/unlabeled_data/*"):
    basename = os.path.basename(ul_path)
    id = basename.split(".")[0]
    path = os.path.join(".", "unlabeled_data", basename)
    
    ids.append(id)
    paths.append(path)

In [ ]:
df = pd.DataFrame({
    "id": ids,
    "path": paths
})

In [ ]:
df.to_csv("unlabled_data.csv", index=False)

In [ ]:
paths[763]